In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


In [38]:
sub=pd.read_csv("../input/animal-classification-challenge/sample_submission (4).csv", sep=",")

In [39]:
sub

,"id,label"
0,"ASG001dw7n_1.jpeg,1"
1,"ASG001dw7q_2.jpeg,0"
2,"ASG001dw7s_1.jpeg,0"
3,"ASG001dw7w_1.jpeg,1"
4,"ASG001dw7x_0.jpeg,0"
...,...
5994,"ASG001g7r0_1.jpeg,"
5995,"ASG001g7r0_2.jpeg,"
5996,"ASG001g7ry_1.jpeg,"
5997,"ASG001g7sb_0.jpeg,"


In [40]:
test=[]
id_=[]
diir="../input/animal-classification-challenge/test/test/"
for i in os.listdir(diir):
    id_.append(i)
    path=diir+i
    test.append(path)
    

In [41]:
train_zebra=[]
diir="../input/animal-classification-challenge/train_zebras/train_zebras/"
for i in os.listdir(diir):
    path=diir+i
    train_zebra.append(path)
    

In [42]:
train_ele=[]
diir="../input/animal-classification-challenge/train_elephants/train_elephants/"
for i in os.listdir(diir):
    path=diir+i
    train_ele.append(path)

In [43]:
z=[1 for i in range(len(train_zebra))]
e=[0 for i in range(len(train_ele))]
L=train_zebra+train_ele
l=z+e


In [44]:
len(l)

13999

In [45]:
train_df=pd.DataFrame({"path":L,"label":l})

In [46]:
train_df

,path,label
0,../input/animal-classification-challenge/train...,1
1,../input/animal-classification-challenge/train...,1
2,../input/animal-classification-challenge/train...,1
3,../input/animal-classification-challenge/train...,1
4,../input/animal-classification-challenge/train...,1
...,...,...
13994,../input/animal-classification-challenge/train...,0
13995,../input/animal-classification-challenge/train...,0
13996,../input/animal-classification-challenge/train...,0
13997,../input/animal-classification-challenge/train...,0


In [47]:
train_df=train_df.sample(frac=1)

In [48]:
train_df

,path,label
7539,../input/animal-classification-challenge/train...,0
3890,../input/animal-classification-challenge/train...,1
10138,../input/animal-classification-challenge/train...,0
4736,../input/animal-classification-challenge/train...,1
6163,../input/animal-classification-challenge/train...,1
...,...,...
9222,../input/animal-classification-challenge/train...,0
10606,../input/animal-classification-challenge/train...,0
7087,../input/animal-classification-challenge/train...,0
1940,../input/animal-classification-challenge/train...,1


In [49]:
train_datagenerator = ImageDataGenerator(
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rescale=1./255, 
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.1)

In [50]:
train_generator = train_datagenerator.flow_from_dataframe(
    train_df, 
    x_col='path', 
    y_col='label',
    target_size=(330, 330), 
    color_mode='rgb', 
    class_mode='raw', 
    batch_size=32, 
    shuffle=True, 
    seed=1,
    subset='training')
validation_generator = train_datagenerator.flow_from_dataframe(
    train_df , 
    x_col='path',
    y_col='label',
    target_size=(330, 330),
    color_mode='rgb', 
    class_mode='raw', 
    batch_size=4, 
    shuffle=True,
    seed=1, 
    subset='validation'
)

Found 12600 validated image filenames.
Found 1399 validated image filenames.


In [51]:
# plt.figure(figsize=(12, 12))
# for i in range(0, 6):
#     plt.subplot(2, 3, i+1)
#     for X_batch, Y_batch in validation_generator:
#         image = X_batch[0]
#         plt.imshow(image)
#         break
# plt.tight_layout()
# plt.show()

In [52]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
model = VGG16(include_top=False, input_shape=(330, 330, 3))

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(2, activation='softmax')(class1)

model = Model(inputs=model.inputs, outputs=output)


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [53]:
batch_size=20
FAST_RUN = False
epochs=5 if FAST_RUN else 20
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/20
394/394 [==============================] - 436s 1s/step - loss: 1.3611 - acc: 0.6353 - val_loss: 0.4912 - val_acc: 0.7684
Epoch 2/20
108/394 [=======>......................] - ETA: 4:48 - loss: 0.5366 - acc: 0.7311

KeyboardInterrupt: 

In [54]:
p=[]
from PIL import Image
for i in test:
    img = np.array(Image.open(i).resize((330,330))).reshape((1, 330, 330 , 3))
    out = model(img)
#     print(out)
#     print(tf.get_static_value(tf.math.argmax(out, axis=1)))
    p.append(tf.get_static_value(tf.math.argmax(out, axis=1)))

In [55]:
tf.get_static_value(tf.math.argmax(out, axis=1))

array([1])

In [56]:
len(id_)

5999

In [57]:
p=[i[0] for i in p]

In [58]:
len(p)

5999

In [60]:
submission=pd.DataFrame({"id":id_,"label":p})
submission.to_csv("sub6.csv", index=False)